# Vorbereitung

In [6]:
%env KERAS_BACKEND=theano
%env THEANO_FLAGS=floatX=float32,device=cpu

env: KERAS_BACKEND=theano
env: THEANO_FLAGS=floatX=float32,device=cpu


In [7]:
%load_ext autoreload
%autoreload 1
%aimport bb_behavior
%aimport bb_behavior.plot
%aimport bb_behavior.tracking
%aimport bb_behavior.tracking.pipeline

import bb_behavior
import bb_behavior.plot
import bb_behavior.tracking
import bb_behavior.tracking.pipeline

import pandas as pd
import time
import datetime

import os
import glob

Using Theano backend.


In [8]:
from tqdm import tqdm_notebook # progress bar

import math
import numpy as np
from bb_tracking.data.constants import DETKEY
#from bb_tracking.tracking import score_id_sim_v
from bb_tracking.tracking import distance_orientations_v, distance_positions_v

from bb_behavior.tracking.pipeline import detect_markers_in_video
from bb_behavior.tracking.pipeline import track_detections_dataframe
from bb_behavior.tracking.pipeline import display_tracking_results

# Run

In [9]:
from bb_behavior.tracking.pipeline import get_default_pipeline
default_pipeline = None
default_pipeline = get_default_pipeline(localizer_threshold="0.50")

/Users/ronnyruhe/.local/lib/python3.7/site-packages/keras/backend/theano_backend.py:1032: UserWarning: DEPRECATION: the 'ds' parameter is not going to exist anymore as it is going to be replaced by the parameter 'ws'.
  mode='max')
/Users/ronnyruhe/.local/lib/python3.7/site-packages/keras/backend/theano_backend.py:1032: UserWarning: DEPRECATION: the 'st' parameter is not going to exist anymore as it is going to be replaced by the parameter 'stride'.
  mode='max')
/Users/ronnyruhe/.local/lib/python3.7/site-packages/keras/backend/theano_backend.py:1032: UserWarning: DEPRECATION: the 'padding' parameter is not going to exist anymore as it is going to be replaced by the parameter 'pad'.
  mode='max')


In [10]:
# Hilfsfunktionen
def filename_to_datestring(filname):
    """
    filename can be path
    """
    return os.path.split(filname)[-1].split('.')[0].split('_')[1]

def datestring_to_filename(datestring, prefix="e00_"):
    return config["videos_dir"] + prefix + datestring + ".h264"

def string_to_timestamp(datestring):
    """ 
    params
        string: format 2018-08-19-01-08-13
    output
        unix timestamp (float)
    """
    
    return time.mktime(time.strptime(datestring, "%Y-%m-%d-%H-%M-%S"))

def timestamp_to_string(timestamp):
    return time.strftime("%Y-%m-%d-%H-%M-%S", time.localtime(timestamp))

In [12]:
# get all videos between timestamp_in and timestamp_out
def get_videos_between(timestamp_in, timestamp_out, all_paths):
    """ returns all video between timestamp_in and timestamp out (inclusive) """
    
    mask = (all_paths['video'] >= timestamp_in) & (all_paths['video'] <= timestamp_out)
    return list(all_paths[mask]['video'])

def loadGTD(path):
    # read in the test data csv
    test_data = pd.read_csv(path)

    # convert the full filenames to string timestamps, and sort by timestamp_in
    test_data['video'] = test_data['video'].apply(lambda x: filename_to_datestring(x))
    test_data['video_start_time'] = test_data['timestamp_in'].apply(lambda x: filename_to_datestring(x))
    test_data['video_end_time'] = test_data['timestamp_out'].apply(lambda x: filename_to_datestring(x))
    test_data.sort_values(['video_start_time'], inplace=True)

    test_data.drop('timestamp_in', 1, inplace=True)
    test_data.drop('timestamp_out', 1, inplace=True)
    test_data.drop('video', 1, inplace=True)
    return test_data

In [13]:
tmp = loadGTD("bees_test.csv")
#tmp

In [14]:
config = dict(tag_pixel_diameter=50,
              n_frames=None,
              confidence_filter_detections=0.08,
              confidence_filter_tracks=0.20,
              coordinate_scale=1.0,
              start_time=None,
              fps=10.0,
              cam_id=0,
              left_leaving_area = 0.3, # Prozente vom Bildschirmrand, zB. bei 1000px und 0.15 -> 0-150px
              right_leaving_area = 0.3,
              px_x_resolution_vid = 1944,
              videos_dir = "../videos/")


In [15]:
def detect_tracks(paths, save_to_csv = False):
    # Hier passiert das eigentliche Tracken und speichern der Ergebnisse:
    num_processed_videos = 0
    video_data = dict()
    frame_info = None
    detections = None
    bad_paths = []

    for path in tqdm_notebook(paths):
        start_time = config["start_time"]
        cam_id = config["cam_id"]
        try:
            num_processed_videos += 1

            frame_info, detections = detect_markers_in_video(path,
                                                          decoder_pipeline=default_pipeline,#pipeline=pipelines(),
                                                         tag_pixel_diameter=config["tag_pixel_diameter"],
                                                          n_frames=config["n_frames"],
                                                          fps=config["fps"],
                                                         progress="tqdm_notebook"
                                                )
            # Sonst würden keine Tracks erkannt werden -> Fehlermeldung
            if len(detections[detections['confidence']>=config["confidence_filter_detections"]]) == 0:
                continue
            tracks = track_detections_dataframe(detections,
                                                tracker="tracker.det_score_fun.frag_score_fun.dill",
                                                confidence_filter_detections=config["confidence_filter_detections"],
                                               confidence_filter_tracks=config["confidence_filter_tracks"],
                                                coordinate_scale=config["coordinate_scale"],
                                               )
            date_string = filename_to_datestring(path)
            tracks['video'] = date_string
            video_data[path] = (frame_info, detections, tracks)
        except ValueError as err: #tritt auf, wenn Video leer ist. In diesem Fall: überspringe video
            try:
                bad_paths.append(path)
                # wir arbeiten später nochmal mit paths, daher müssen das leere löschen, weil sonst
                # in video_data kein zugehöriger Value zu Key = file zu finden ist.
            except KeyError as err:
                continue
        except Exception as err:
            print(err)
            raise
        # only first vid: break

    for bad_path in bad_paths:
        paths.remove(bad_path)

    #video_data["file"][0] --> frame-info, [1] --> detections, [2] --> tracks
    for path in paths:
        display_tracking_results(path, video_data[path][0], video_data[path][1], video_data[path][2])

    tracks = [video_data[paths[x]][2] for x in range(len(paths))]
    tracks = pd.concat(tracks,ignore_index=True)
    tracks = tracks.drop(columns=["localizerSaliency", "beeID", "camID", "frameIdx"])

    if save_to_csv:
        with open("tracks.csv", "w") as f:
            tracks.to_csv(f)

    return tracks

## a: Create video path list from all videos in videos_dir

In [16]:
# Einen Iterable speichern, der alle Videos in einem Iterable zur Verfügung stellt
# Diesen Iterable können wir dann in der nächsten Zelle mit tqdm schön durchlaufen
base_directory = config["videos_dir"]
paths = [i for i in os.listdir(base_directory) if i.endswith(".h264")]
for i in range(len(paths)):
    paths[i] = base_directory + paths[i]

## b: Create video path list from test set

In [17]:
# read in the test data csv
test_data = loadGTD("bees_test.csv")

all_videos = []

all_paths = pd.DataFrame(glob.glob(os.path.join(config["videos_dir"], '*.h264')), columns=['video'])
all_paths['video'] = all_paths['video'].apply(lambda x: filename_to_datestring(x))
all_paths.sort_values(['video'])

# go through test_data and get all videos between timestamp_in and timestamp_out
for index, row in test_data.iterrows():
    all_videos += get_videos_between(row['video_start_time'],row['video_end_time'], all_paths)
    
all_videos = sorted(list(set(all_videos)))
paths = [datestring_to_filename(x) for x in all_videos]
del all_paths
del test_data

# Continue here

## Zwischenschritt: Merge all close Tracks of one bee

In [18]:
# RUN THIS IF TRACKS WHERE ALREADY CALCULATED AND SAVED TO AN .csv
# CSV EINLESEN UND SETZEN
tracks = pd.read_csv("all_tracks.csv", sep=";")

In [19]:
def gather_tracks(tracks):
    """
    transform the tracks df to : bee_id, [list of positions (x,y)], [list of timestamps], timestamp_of video
    """

    #  transform tracks df to : bee_id, [list of positions (x,y)], [list of timestamps], timestamp_of video
    grouped = tracks.groupby(['bee_id','track_id','video'],  as_index=False)['xpos','ypos','timestamp','zrotation']

    tracks_ml = grouped.aggregate(lambda x: list(x))

    # we don't need track_id anymore
    tracks_ml = tracks_ml.drop('track_id', 1)

    # add a column: convert video name to timestamp
    tracks_ml['video_start_time'] = tracks_ml['video'].apply(lambda x: string_to_timestamp(x))
    
    # we don't need video anymore
    tracks_ml = tracks_ml.drop('video', 1)

    # because aggregated: now multiple timestamps per row: --> rename
    tracks_ml = tracks_ml.rename(columns={'timestamp': 'timestamps'})

    # calculate start time of track by adding timestamp of track (seconds since start of video)
    # to timestamp of video (date)
    tracks_ml['track_start_time'] = tracks_ml['video_start_time'] + tracks_ml['timestamps'].apply(lambda x: x[0])
    tracks_ml['track_end_time'] = tracks_ml['video_start_time'] + tracks_ml['timestamps'].apply(lambda x: x[-1])
    
    # convert back to string
    tracks_ml['video_start_time'] = tracks_ml['video_start_time'].apply(lambda x: timestamp_to_string(x))
    
    all_paths = pd.DataFrame(glob.glob(os.path.join(config["videos_dir"], '*.h264')), columns=['video'])
    all_paths['video'] = all_paths['video'].apply(lambda x: filename_to_datestring(x))
    all_paths.sort_values(['video'])
    # get end video
    tracks_ml['video_end_time'] =  tracks_ml['track_end_time'].apply(lambda x: timestamp_to_string(x))
    tracks_ml['video_end_time'] =  tracks_ml[['video_start_time','video_end_time']].apply(lambda x: get_videos_between(x[0],x[1],all_paths)[-1], axis=1)

    return tracks_ml

def merge_tracks(tracks_ml, verbose = False):
    """
    in: 
        tracks_ml: output from gather_tracks(tracks)
    out:
        same df as in, with merged rows
        [bee_id:Float, xpos:[Float], ypos:[Float], zrotation:[Float], timestamps:[Float], video_start_time: String,
        video_end_time: String, track_start_time:Float, track_end_time:Float]
        
    merge tracks of same bee where start and end timestamps are close together
    assume there can not be overlapping tracks
    """
    
    
    # 1. sort: bee_id, start_time
    tracks_ml = tracks_ml.sort_values(['bee_id', 'track_start_time'])

    
    # first convert to timestamp
    tracks_ml['video_start_time'] = tracks_ml['video_start_time'].apply(lambda x: string_to_timestamp(x))
    
    index = 0
    while(True):
        row = tracks_ml.iloc[index]
        next_row = tracks_ml.iloc[index+1]

        # if the tracks are from same bee and the start time of next row is closer then 10s -> merge the rows
        # merge rows means, next_row is deleted
        if (row['bee_id'] == next_row['bee_id']) and ((next_row['track_start_time'] - row['track_end_time']) < 10):
            
            if verbose:
                print(timestamp_to_string(row['video_start_time']), row['bee_id'], next_row['track_start_time'] - row['track_end_time'])
                print(next_row['video_start_time'] - row['video_start_time'])
                print(timestamp_to_string(row['track_start_time']),timestamp_to_string(row['track_end_time']),timestamp_to_string(row['video_start_time']),row['video_end_time'])
                print(row['timestamps'])
                print(row.name)
                print(timestamp_to_string(next_row['track_start_time']))
                print('----------------------------------')
           
            # update the timestamps of nextrow
            t = next_row['video_start_time'] - row['video_start_time']
            timestamps = list( np.array(next_row['timestamps']) + t)

            # merge xpos, ypos, timestamps lists
            tracks_ml.at[row.name,'xpos'] = row['xpos']+next_row['xpos']
            tracks_ml.at[row.name,'ypos'] = row['ypos']+next_row['ypos']
            tracks_ml.at[row.name,'timestamps'] = row['timestamps']+timestamps

            # update end_time
            tracks_ml.at[row.name,'track_end_time'] = next_row['track_end_time']
            tracks_ml.at[row.name,'video_end_time'] = next_row['video_end_time']

            # delete the merged row (next_row)
            tracks_ml = tracks_ml.drop(next_row.name).copy()

        else:
            index += 1
        
        if index == len(tracks_ml) - 1:
            break
            
    # convert back to string
    tracks_ml['video_start_time'] = tracks_ml['video_start_time'].apply(lambda x: timestamp_to_string(x))
            
    return tracks_ml

In [20]:
tracks_ml = gather_tracks(tracks)
tracks_ml = merge_tracks(tracks_ml)
tracks_ml = tracks_ml.sort_values(['video_start_time'])

In [21]:
# read in the test data csv
test_data = loadGTD("bees_test.csv")

# get all tracks where bee_id, videio_start_time and video_end_time are the same
tracks_ml = pd.merge(tracks_ml,test_data,how='inner',on=['bee_id','video_start_time','video_end_time'])

# we only look at tracks where  in_direction and out_direction are different
tracks_ml = tracks_ml[(tracks_ml['in_direction'] != tracks_ml['out_direction'])]

# we assign movement based on in_ and out_directoin
tracks_ml.loc[(tracks_ml['in_direction'] == 'left') & (tracks_ml['out_direction'] == 'right'),'movement'] = 1
tracks_ml.loc[(tracks_ml['in_direction'] == 'right') & (tracks_ml['out_direction'] == 'left'),'movement'] = 0

tracks_ml.drop('in_direction', 1, inplace=True)
tracks_ml.drop('out_direction', 1, inplace=True)

# Baseline-Algorithmen

## Algorithmus 1: Baseline - Areas as Decider

In [22]:
from math import pi
""" Vorgehen:
Für jedes Bienen-Track Paar
- Prüfe, wo sich die Biene beim ersten erkannten Erscheinen aufhält
    - Ordne die Position in left, right oder middle ein
- Prüfe, wo sich die Biene beim letzten erkannten Erscheinen aufhält
    - Ordne die Position in left, right oder middle ein
Wenn sich Biene in der Mitte befindet, starte neue Routine, die links oder rechts zuordnet
Daraus kann nun abgeleitet werden, wo die Biene reingekommen ist und wo sie rausgegangen ist.
"""
# param: tracks_ml, s. oben
def baseline_alg_classify_bee(tracks_ml):
    def get_direction(xpos, zpos):
        #Helpers
        def is_left(xpos):
            return xpos <= config["left_leaving_area"]*config["px_x_resolution_vid"]
        def is_right(xpos):
            return xpos >= config["px_x_resolution_vid"] - config["right_leaving_area"]*config["px_x_resolution_vid"]
        # Routine, wenn Biene in der Mitte
        # Betrachte zRotation, das ist die Richtung in die die Biene guckt (in Bogenmaß)
        # gebe diese Richtung aus
        def middle_classifier(zRotation):
            if abs(zRotation) > pi/2:
                return "left"
            else:
                return "right"
            
        # Eintrittsseite festlegen
        if is_left(xpos):
            return "left"
        elif is_right(xpos):
            return "right"
        else:
            return middle_classifier(zpos)
        
    pred_in_direction = len(tracks_ml) * [None]
    pred_out_direction = len(tracks_ml) * [None]
    i = 0
    for index, row in tracks_ml.iterrows():
        pred_in_direction[i] = get_direction(row['xpos'][0], row['zrotation'][0])
        pred_out_direction[i]= get_direction(row['xpos'][-1], row['zrotation'][-1])
        i += 1
    
    #remove those with no 
    return pd.DataFrame({"pred_in_direction":pred_in_direction, "pred_out_direction":pred_out_direction})

## Algorithmus 2: Baseline - zpos as decider

In [ ]:
from math import pi
""" Vorgehen:
Unterschied: Hier wird nicht auf die Position abgestellt, sondern ausschließlich auf die
Richtung, in die die Biene schaut
"""
#results = {"bee_id":[], "time_in":[], "os_in":[], "time_out":[], "os_out":[], "in_direction":[], "out_direction":[]}
# param: tracks_ml, s. oben
def baseline_2_alg_classify_bee(tracks_ml):
    def get_in_direction(zpos):
        # Betrachte zRotation, das ist die Richtung in die die Biene guckt (in Bogenmaß)
        # gebe diese Richtung aus
        if zpos > abs(pi/2):
            return "right"
        else:
            return "left"
    def get_out_direction(zpos):
        # Betrachte zRotation, das ist die Richtung in die die Biene guckt (in Bogenmaß)
        # gebe diese Richtung aus
        if zpos > abs(pi/2):
            return "left"
        else:
            return "right"
    
    # "bee_id":[int], "time_in":[String], "os_in":[Float], "time_out":[String],
    # "os_out":[Float], "in_direction":[{"left", "right"}], "out_direction":[{"left", "right"}]
    results = {"bee_id":[], "time_in":[], "os_in":[], "time_out":[], "os_out":[], "in_direction":[],
               "out_direction":[]}
        
    for row in tracks_ml.itertuples(index=False):
        results["bee_id"].append(row[0])
        results["time_in"].append(row[5])
        results["os_in"].append(row[-3] - string_to_timestamp(row[5])) #track_start_time - video_start_time
        results["time_out"].append(row[-1])
        results["os_out"].append(row[-2] - string_to_timestamp(row[-1])) #track_end_time - video_end_time
        results["in_direction"].append(get_in_direction(row[4][0]))
        results["out_direction"].append(get_out_direction(row[4][-1]))
    
    return results
    

# Algorithmus 3: Machine Learning

## Vorbereiten Lernvideos, Prüfvideos

### Optional: Tracks um 180° drehen; Tracks horizontal invertieren; Tracks vertikal invertieren (Funktioniert, abhängig von der Wichtigkeit der Zeit als Feature, vielleicht nicht!)

In [ ]:
# Tracks um 180° drehen
track_rotated = tracks_ml


## 3.1 Lernen Logistische Regression

## 3.1 Alg: Logistische Regression

## 3.2 Lernen Decision Tree

## 3.2 Alg Decision Tree

# Vergleichsfunktion Algorithmus mit Labels

In [23]:
""" 
Input: bees_test.csv, MachineLearnedData.csv
Output: "result" as DataFrame with an overview of compareable bees and their correct/wrong correctness

Read in two csv file and store values in DataFrame's. Merge DataFrame's to get exact correctness of 
total-, in- and out_direction of bees regarding their time_in and time_out relation. 
"""

# store data in dataFrame
total_gtd = loadGTD('bees_test.csv')
gtd = total_gtd.copy()

# drop dublicates
gtd = gtd.drop_duplicates(['bee_id', 'video_start_time', 'video_end_time'], keep='first')

# merge with our tracking output
gtd = pd.merge(gtd, tracks_ml, on=['bee_id', 'video_start_time', 'video_end_time'], how='inner')

# get result from baseline algorithm
mld = baseline_alg_classify_bee(gtd)

# iterate mergeableBees for visualization
result = pd.DataFrame()
result['bee_id'] = gtd['bee_id'] 
result['in_equals'] = mld['pred_in_direction'] == gtd['in_direction']
result['out_equals'] = mld['pred_out_direction'] == gtd['out_direction']

# store to result.csv
result.to_csv('result.csv', encoding='utf-8', index=False)

# calculate statitics
inCorrect = round((result['in_equals'].sum()/len(result) * 100), 2)
outCorrect = round((result['out_equals'].sum()/len(result) * 100), 2)
allCorrect = round(((result['in_equals'] == result['out_equals']).sum()/len(result) * 100), 2)
aveCorrect = round(((allCorrect + inCorrect + outCorrect) / 3), 2)

# print statistics
try:
    print(len(total_gtd), 'entries in GroundTruthData.', len(gtd), 'entries compareable in MachineLearnedData: ', round((len(gtd)/len(total_gtd))*100, 2), '%' )
    print(allCorrect, "% total correctness.")
    print(inCorrect, "% Ingoing correctness.")
    print(outCorrect, "% Outgoing correctness.")
    print(aveCorrect, "% Average correctness.")
    print("Accuracy of ",round((len(gtd)/len(total_gtd))*100*(allCorrect/100),2),"%.")
except ZeroDivisionError:
    print("Nothing to compare!")

# end compare algorithm
print('\nCheck result.csv file or hit \'result\' for more details.')
print('Use DataFrames for deeper information: \n\'mergeTotalCorrect\',\'mergeInCorrect\',\'mergeOutCorrect\',\'mergeableBees\' ')

108 entries in GroundTruthData. 42 entries compareable in MachineLearnedData:  38.89 %
71.43 % total correctness.
78.57 % Ingoing correctness.
92.86 % Outgoing correctness.
80.95 % Average correctness.
Accuracy of  27.78 %.

Check result.csv file or hit 'result' for more details.
Use DataFrames for deeper information: 
'mergeTotalCorrect','mergeInCorrect','mergeOutCorrect','mergeableBees' 


# MachineLearnedAlgotithm

In [112]:
from sklearn.metrics import confusion_matrix

from sklearn.ensemble import RandomForestClassifier
from sklearn.datasets import make_classification

def machineLearning(gtd, pD):
    """ 
    Input: 
            GroundTruthData and MachineLearnedData
    Output: 
            Confusion Matrix
    
    Re-label Input and put information into confusion_matrix method:   
    left left -> 0
    left right -> 1
    right left -> 2
    right right -> 3
    """
    
    # grap neede columns from gtd
    trueData = gtd.copy()
    trueData = trueData[['bee_id','in_direction','out_direction']]
    
    predData = pD.copy()
    
    # combine directions to one output
    def getValue(x,y):
        if (x == 'left' and y == 'left'):
            return 0
        elif (x == 'left' and y == 'right'):
            return 1
        elif (x == 'right' and y == 'left'):
            return 2
        else:
            return 3
    
    # insert reduced information to dataframe
    trueData['reduced'] = [int(getValue(x,y)) for (x,y) in zip(trueData['in_direction'], trueData['out_direction'])]
    predData['reduced'] = [int(getValue(x,y)) for (x,y) in zip(predData['pred_in_direction'], predData['pred_out_direction'])]
    
    # confusion Matrix
    def computeConfusionMatrix(trueData, predData):
        return confusion_matrix(trueData['reduced'], predData['reduced'])

    def computeRandomForest(trueData, predData):
        # use predData['reduced'] as training and test data
        train, test = predData[predData['reduced'] <= 2], predData[predData['reduced'] > 2]
        
        print('Number of observations in the training data:', len(train))
        print('Number of observations in the test data:',len(test))
        
        y = pd.factorize(train['reduced'])[0]
        print(y)
        
        features = predData.columns[0:2]
        clf = RandomForestClassifier(n_jobs=2, random_state=0)
        clf.fit(train[features], predData['reduced'])
        
    print (computeConfusionMatrix(trueData, predData))
    print (computeRandomForest(trueData, predData))


In [113]:
#gtd = mergeTotalCorrect
print(machineLearning(gtd, mld))

[[ 0  0  0  0]
 [ 3 17  0  5]
 [ 4  0 13  0]
 [ 0  0  0  0]]
Number of observations in the training data: 37
Number of observations in the test data: 5
[0 0 1 1 2 0 2 2 0 0 1 1 1 0 0 0 0 0 2 1 2 0 0 0 0 1 1 0 2 1 0 1 1 1 0 1 2]


/Users/ronnyruhe/anaconda3/lib/python3.7/site-packages/sklearn/ensemble/forest.py:246: FutureWarning: The default value of n_estimators will change from 10 in version 0.20 to 100 in 0.22.
  "10 in version 0.20 to 100 in 0.22.", FutureWarning)


ValueError: could not convert string to float: 'left'